In [1]:
import pandas as pd
from ALLCools.clustering import one_vs_rest_dmg
import glob as glob
import anndata


In [2]:
mcds_paths = 'output/geneslop2k-vm23_frac.mcds'
cell_meta_path = 'output/merged-all-pfc-annotated.csv'
cluster_col = 'celltype.L1'

obs_dim = 'cell'
var_dim = 'geneslop2k-vm23'
mc_type = 'CHN'

top_n = 1000
auroc_cutoff = 0.8
adj_p_cutoff = 0.001
fc_cutoff = 0.8
max_cluster_cells = 2000
max_other_fold = 5
cpu = 10


In [3]:
cell_meta = pd.read_csv(cell_meta_path, index_col=0)
cell_meta.head()


/tmp/ipykernel_1223554/96884813.py:1: DtypeWarning: Columns (1,2,17,18,19,20,21,22,23,25,26,31,32,36,37,41,42,43,44,45,46,47,56,85) have mixed types. Specify dtype option on import or set low_memory=False.
  cell_meta = pd.read_csv(cell_meta_path, index_col=0)


,sample,group,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,...,umap_0,umap_1,umap_seurat_0,umap_seurat_1,celltype.L1,Neurotransmitter,Neurotransmitter_celltype,celltype.L2,celltype.L2.p,celltype.L1.p
AAACCCAAGCTAGCCC-1-0-0-1-0,W3,W,847.0,6.742881,1343.0,7.203405,29.560685,38.644825,51.824274,74.162323,...,-3.501324,6.106640,NaN,NaN,Neuron,NaN,GABAergic,Pvalb GABA,Pvalb GABA,Neuron
AAACCCAAGCTGGAGT-1-0-0-1-0,W3,W,2367.0,7.769801,5585.0,8.628018,29.597135,37.887198,47.090421,61.772605,...,-0.719556,1.394005,NaN,NaN,Neuron,NaN,Glutamatergic,L2/3 IT,L2/3 IT,Neuron
AAACCCAAGGATACCG-1-0-0-1-0,W3,W,1975.0,7.588830,3909.0,8.271293,26.502942,34.177539,43.975441,60.629317,...,-4.900810,5.172145,NaN,NaN,Neuron,NaN,Glutamatergic,L6 IT,L6 IT,Neuron
AAACCCAAGGGTCACA-1-0-0-1-0,W3,W,1625.0,7.393878,2899.0,7.972466,25.974474,33.563298,43.428769,61.193515,...,-0.761516,11.300124,NaN,NaN,Neuron,NaN,Glutamatergic,L6 CT,L6 CT,Neuron
AAACCCAAGTGCTAGG-1-0-0-1-0,W3,W,1296.0,7.167809,2210.0,7.701200,27.556561,35.429864,45.701357,63.981900,...,-1.046882,0.096229,NaN,NaN,Neuron,NaN,Glutamatergic,L4/5 IT,L4/5 IT,Neuron


In [4]:
mch_meta = cell_meta[cell_meta.batch == 'mch'].dropna(axis=1, how='all')

In [5]:
mch_meta.set_index('cell', inplace=True)
mch_meta["celltype.L1"].fillna('Unclassified', inplace=True)

In [6]:
dmg_table_chn = one_vs_rest_dmg(mch_meta,
                            group=cluster_col,
                            mcds_paths=mcds_paths,
                            obs_dim=obs_dim,
                            var_dim=var_dim,
                            mc_type=mc_type,
                            top_n=top_n,
                            adj_p_cutoff=adj_p_cutoff,
                            fc_cutoff=fc_cutoff,
                            auroc_cutoff=auroc_cutoff,
                            max_cluster_cells=max_cluster_cells,
                            max_other_fold=max_other_fold,
                            cpu=cpu)

Calculating cluster Astro-Epen DMGs.
Calculating cluster Immune DMGs.
Calculating cluster Neuron DMGs.
Calculating cluster OPC-Oligo DMGs.
Calculating cluster Unclassified DMGs.
Calculating cluster Vascular DMGs.


/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/ALLCools/clustering/dmg.py:370: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dmg_result["AUROC"] = pd.Series(auroc)


Immune Finished.
Astro-Epen Finished.
Vascular Finished.
OPC-Oligo Finished.
Unclassified Finished.
Neuron Finished.


In [7]:
mcg_meta = cell_meta[cell_meta.batch == 'mcg'].dropna(axis=1, how='all')
mcg_meta.set_index('cell', inplace=True)
mcg_meta["celltype.L1"].fillna('Unclassified', inplace=True)

In [8]:
dmg_table_cgn = one_vs_rest_dmg(mcg_meta,
                            group=cluster_col,
                            mcds_paths=mcds_paths,
                            obs_dim=obs_dim,
                            var_dim=var_dim,
                            mc_type="CGN",
                            top_n=top_n,
                            adj_p_cutoff=adj_p_cutoff,
                            fc_cutoff=fc_cutoff,
                            auroc_cutoff=auroc_cutoff,
                            max_cluster_cells=max_cluster_cells,
                            max_other_fold=max_other_fold,
                            cpu=cpu)

Calculating cluster Astro-Epen DMGs.
Calculating cluster Immune DMGs.
Calculating cluster Neuron DMGs.
Calculating cluster OPC-Oligo DMGs.
Calculating cluster Unclassified DMGs.
Calculating cluster Vascular DMGs.


/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/ALLCools/clustering/dmg.py:370: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dmg_result["AUROC"] = pd.Series(auroc)


Immune Finished.
Vascular Finished.
Astro-Epen Finished.
Unclassified Finished.
OPC-Oligo Finished.
Neuron Finished.


In [9]:
dmg_table_chn.to_hdf(f'output/{cluster_col}.OneVsRestDMG.hdf', key='data')


In [11]:
dmg_table_chn

,pvals_adj,fc,AUROC,cluster
names,,,,
ENSMUSG00000004698,3.864956e-18,0.579181,0.919822,Vascular
ENSMUSG00000022883,3.864956e-18,0.437439,0.919067,Vascular
ENSMUSG00000045083,2.083544e-17,0.595792,0.908642,Vascular
ENSMUSG00000028519,2.083544e-17,0.554315,0.908505,Vascular
ENSMUSG00000027674,2.401630e-17,0.322192,0.906859,Vascular
...,...,...,...,...
ENSMUSG00000018654,1.450101e-76,0.641717,0.812335,Unclassified
ENSMUSG00000069170,1.157531e-75,0.779697,0.810260,Unclassified
ENSMUSG00000046782,1.474002e-75,0.711752,0.810005,Unclassified


In [10]:
adata_path = 'output/merged-all-pfc-annotated.h5ad'
adata = anndata.read_h5ad(adata_path)


In [12]:
adata_mch = adata[adata.obs.batch == 'mch']

In [17]:
"ENSMUSG00000016256" in adata_mch.var.index

True

In [ ]:
adata_mch.obs.dropna(subset=[cluster_col], inplace=True)

In [ ]:
adata_mch.obs = adata_mch.obs.dropna(axis=1, how='all')
adata_mch.obs.set_index('cell', inplace=True)